# Astropy

In this notebook we will introduce Astopy. This is a module that has numerous helper functions aimed at practicing astronomers - but is also useful in the broader physics environment.

We'll look at just a few of the areas where Astropy comes into its own - **units** which allows quick and accurate specification and conversion between scientific units, **constants** which gives acces to a huge number of physics and engineering constants, and **Coordinates** which facilitates transfer from one astronomical coordinate system to another - we'll look at converting between Galactic and Celestial coordinates. 

As always we start off by importing the modules - in this case we only import 3 functions from the astropy.coordinates package, but all the units and constants packages.


In [2]:
from astropy import constants as const
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time

## 1.  Units and Quantity

We'll start with 'units'. As an example to demonstrate the features this module has we'll enter the distance to the Andromeda nebula in light years (2537000) and then convert this to parsecs, metres and astronomical units.

There are a large number of useful units defined in astropy.units. Go to the astropy ['Units and Quantities'](http://docs.astropy.org/en/stable/units/) to get a feel for these.

First, we need to enter our value as an astropy 'Quantity'. The easy way to do this is to enter a nummeric value multiplied by one of the appropriate astropy built-in units. (This works with numpy and Pandas data structures as well as single values.) Remember, we've imported astropy.units as 'u'.


In [3]:
andromeda_d = 2.537e6*u.lyr

print(andromeda_d)
# What internal 'type' is this then?
print(type(andromeda_d))

2537000.0 lyr
<class 'astropy.units.quantity.Quantity'>



Now it's easy to see this in any other unit using the **'.to'** method and specifying the new unit in brackets. Note that the units are added in the print string. Easiest to see by example:


In [4]:
print('In parsecs: ', andromeda_d.to(u.pc))
print('In metres: ', andromeda_d.to(u.m))
print('In kilometres: ', andromeda_d.to(u.km))
print('In Astronomical Units: ', andromeda_d.to(u.au))

In parsecs:  777847.736040036 pc
In metres:  2.400187320893749e+22 m
In kilometres:  2.4001873208937492e+19 km
In Astronomical Units:  160442612562.78357 AU



So, it's simple to enter a quantity in an easy, human understandable form (like light years), convert to the appropriate SI unit (metres in this case), do the calculations you need and return back to the comportable light years at the end.


### Excercise 1

Find out how to enter a pressure of 1013.25 hectopascals (hPa - this is 'standard' atmospheric pressure) and convert this to pounds per square inch (psi) - should be around 15psi. 

Hint a 'hectopascal' is 100 Pascals. You could do the conversion manually but astropy.units allows the use of [standard prefixes](http://docs.astropy.org/en/stable/units/standard_units.html).

Hint: For the last unit (psi) you'll have to use the 'imperial' module - part of 'units'.


In [14]:
std_p = 1013.25*u.hPa
#print(std_p)
#print (type(std_p))

imp_std_p = std_p.to(u.imperial.psi)

print('Standard atmospheric pressure is', imp_std_p)


Standard atmospheric pressure is 14.695948572906829 psi



Just for information, we can print this result with, say 2 decimal places and 2 significant figures:


In [15]:
print(f'Standard atmospheric pressure (2 dp) is {imp_std_p:.2f}')
print(f'Standard atmospheric pressure (2 sf) is {imp_std_p:.2g}')

Standard atmospheric pressure (2 dp) is 14.70 psi
Standard atmospheric pressure (2 sf) is 15 psi



## 2. Constants

Astropy has a load of built-in physical constants. Have a look at the [astropy.constants](https://astropy.readthedocs.io/en/stable/constants/index.html) pages for details, but here are some simple examples (Note they are in 'Quantity' units as we've just seen):

In [16]:
print('Speed of light (in a vacuum)  is', const.c)
print('The gravitation constant is', const.G)
print('The standard atmosphere is', const.atm)

Speed of light (in a vacuum)  is   Name   = Speed of light in vacuum
  Value  = 299792458.0
  Uncertainty  = 0.0
  Unit  = m / s
  Reference = CODATA 2014
The gravitation constant is   Name   = Gravitational constant
  Value  = 6.67408e-11
  Uncertainty  = 3.1e-15
  Unit  = m3 / (kg s2)
  Reference = CODATA 2014
The standard atmosphere is   Name   = Standard atmosphere
  Value  = 101325
  Uncertainty  = 0.0
  Unit  = Pa
  Reference = CODATA 2014



You can, of course get the actual value:


In [17]:
print('Speed of light (in a vacuum)  is', const.c.value)

Speed of light (in a vacuum)  is 299792458.0



### Using Constants in calculations.

Let's work out the energy in 1 gram of matter, expressed as giga Joules. 

Note how easy it is to convert between unit magnitudes.

Also note we've expressed the answer in scientific notation to 3 decimal places
    

In [20]:
E = ((1.0*u.g)*(const.c)**2).to('GJ')

print(f'One gram of matter contains {E:.3e}')

One gram of matter contains 8.988e+04 GJ



### Exercise 2

Work out the gravitational force betwen the Sun and the Earth using Newtons gravitational equation $G{m_1m_2} \over {d^2}$, express it as mega Newtons.

Hint: All the values you need are defined in constants.

Hint: The distance from Earth to Sun is one Astronomical Unit (AU)


In [25]:
f = const.G * const.M_earth*const.M_sun/const.au**2
fmn = f.to('MN')
print(f'The force between the Earth and the Sun is: {fmn:.2e}')

The force between the Earth and the Sun is: 3.54e+16 MN


## 3. Coordinate systems

Now we'll look at specifying an astronomical spacial position in one coordinate system (often refered to as a 'frame') and converting this to another.

The Sun is generally moving with a speed of around 20km/s towards the bright star Vega in the constellation of Lyra. If you're doing the ARROW topic, you'll discover later that when measuring radial velocities to objects within our galaxy, we often need to correct for this motion of the Sun - correcting to the Local Standard of Rest (LSR). 

For this example we're given the position of Vega, in Galactic coordinates (galactic longitude, l=55.8585 degrees, and latitude, b=23.5489 degrees) but need these in celestial equatorial coordinate values of Right ascension and declination (Ra/Dec).

First we need to specify the l,b position as an astropy 'SkyCoord', giving the l and b values and also the 'coordinate frame' - in this case it's a Galactic frame specified by the parameter 'galactic'.

There a number of different frames we could convert to (have a look at the Astropy  ['Astronomical Coordinate Systems'](http://docs.astropy.org/en/stable/coordinates/) pages) but here the one we need is 'icrs'.


In [28]:
vega_lb = SkyCoord(l=55.8585*u.deg, b=23.5489*u.deg, frame='galactic')

We've specified the l and b units separately using astropy units. We could also use the form:

In [29]:
vega_lb = SkyCoord(l=55.8585, b=23.5489, frame='galactic', unit='deg')


As is often the case there is more than one way to get these coordinates coverted to another **frame** - in our case from 'galactic' to 'icrs'.

Here are 2 such methods:


In [30]:
vega_radec = vega_lb.icrs
print(vega_radec)

vega_radec_icrs = vega_lb.transform_to('icrs')
print(vega_radec_icrs)

<SkyCoord (ICRS): (ra, dec) in deg
    (270.00003058, 30.00018541)>
<SkyCoord (ICRS): (ra, dec) in deg
    (270.00003058, 30.00018541)>


In [31]:
from astropy.coordinates import FK5
vega_radec_fk = vega_lb.transform_to(FK5(equinox='J1975'))
print(vega_radec_fk)

<SkyCoord (FK5: equinox=J1975.000): (ra, dec) in deg
    (269.76013133, 30.00048048)>



We can easily get the RA and Dec values:
    

In [32]:
print('RA is', vega_radec.ra)
print('RA, in degress, is', (vega_radec.ra)*u.deg)

RA is 270d00m00.1101s
RA, in degress, is 270.00003057723364 deg2


### Exercise 3

What's the Ra/Dec of the centre of the Galaxy? We'll leave this up to you to complete.

Hint: What are the 'l' and 'b' coordinates of the Galactic centre?



### 2.1 Doing calculations with coordinates

Astropy also does spatial and angular calculations. Let's look at the Large and Small Magellanic clouds. These are small, irregular galaxies in our Local Group of galaxies. They are so called because they can bee seen as vague smudges of light in the Southern skies looking somewhat like dimly illuminated rain clouds and, reputedly, Magellan (the great 16tn century Portuguese explorer) thought that's what they were.

Anyway, the Large Magellanic Cloud (LMC) lies at RA=05h23m34.5s, Dec=-69:45:22 at a distance of 163 kly and the Small Magellanic Cloud (SMC) at RA=00h52m44.8s, Dec=-72:49:43 and 200kly.

How far apart are they - in angular terms from the Earth and actual distance?

By example, we'll introduce here new ways of specifying coordinates, including distances, and show how to do the calculations.



In [33]:
lmc = SkyCoord('05h23m34.5s', '-69d45m22s', distance = 163*u.klyr)
smc = SkyCoord('00h52m44.8s', '-72d49m43s', distance = 200*u.klyr)

print('The LMC is separated from the SMC by', lmc.separation_3d(smc))
print('and by', lmc.separation(smc), 'on the sky')


The LMC is separtaed form the SMC by 74.81055774428279 klyr
and by 20d44m46.0264s on the sky


### Exercise 3.1

Look up the stars Rigel and Betelgeuse (Opposite sides of the Orion constellation - Wikipedia will give you all the information you need) and compute how far apart they are in space. 

At 0.9 times the speed of light, how long would it take to get from one to the other (ignore any relativistic effects).



## 4 Earth location coordinates

Most observations will be made on Earth based telescopes. For these instruments we are interested in the direction it is pointing - specified by the Altitude and Azimuth angles (Alt/Az). And remember, as the Earth moves during an observation, the Alt and Az values will be continually changing - so the time is also important.

How do we specify or calculate these values? We'll look at specifying the location of the PIRATE and ARROW telescopes and an observing time of 21h58m00s (UTC) on 20st March 2019

ARROW is at 52.0244W, 0.70639W, at an altitude of 115m. PIRATE is at 28.3N, 16.5097W at an altitude of 2390m.


###  4.1 Time

This is pretty easy to do by concatenating a date and time string and giving this to the [astropy Time](http://docs.astropy.org/en/stable/time/) method. 

Note: 'format='isot' indicates the format of the input string, 'scale='utc' notes the time scale. If the string format is unambiguous these parameters can be left out.


In [36]:
time_str= '2019-03-20T21:58:00'
obs_time = Time(time_str, format='isot', scale='utc')
print(obs_time)

2019-03-20T21:58:00.000


### 4.2 Location

Once again, pretty self explanatory. 

Note the use of '\\' here to indicate that the command continues on the next line. This is not obligatory - we could just use a long single line - but it does make the code easier to read.

Note, a negative sign indicates West (or South).


In [37]:
arrow = EarthLocation(lat=52.024444*u.deg, \
                   lon=-0.706388*u.deg, \
                   height=114*u.m)

pirate = EarthLocation(lat=28.3*u.deg, \
                   lon=-16.5097*u.deg, \
                   height=2390*u.m)



### 4.3 Frames

Earlier, when converting coordinate systems we used differing 'frames - GALACTIC, ICRS, FK5. These were indicated by a simple string, for example 'icrs' which are recognised by astropy. If we want to get to an Alt/Az frame we need to provide further information - an internal 'string' cannot know the date, time or location of our telescope. This is done using the AltAz() method.

Using the time and locations we described above:


In [38]:
obs_frame = AltAz(obstime=obs_time , location=arrow )


Now we can convert from our previously defined Vega coordinates in the icrs frame (vega_radec_icrs) to our new, arrow specific altAz frame using the '**transform_to method**' - like this:


In [39]:
obs_altaz = vega_radec_icrs.transform_to(obs_frame)


You can get at the altitude and azimuth values (in degrees) like this:


In [40]:
print ('Altitude is',obs_altaz.alt.deg, 'degrees')
print ('Azimuth is',obs_altaz.az.deg, 'degrees')

Altitude is 5.8495137194389635 degrees
Azimuth is 46.724733745586 degrees


### Exercise 4.3

Work out what 'Alt' values you would have needed to point a telescope located at the  PIRATE location towards Polaris, the Pole Star, (well, one of them anyway) at 20h17m00s (UTC) on 20th July 1969. (Any guesses as to why this date/time is interesting?).

Do the same for the LMC (information earlier)

What does this tell you about the visibility of the two targets at the time - what does a negative 'Alt' value indicate?

Hint: This is pretty much the same as our previous coordinate conversions. Here you will be going from RA/Dec to AltAz and this will, as before, require a location and time as extra information.

Hint: As earlier, use the .transform_to() form - but instead of a string, you'll need to pass it more complete 'frame' information.


In [45]:
pol=SkyCoord('02h30m41s', '89d15m38s')

p_str='1969-07-20T20:17:00'
p_time = Time(p_str, format='isot', scale='utc')
p_frame=AltAz(obstime=p_time, location=pirate)

p_altaz=pol.transform_to(p_frame)
lmc_altaz=lmc.transform_to(p_frame) #lmc SkyCoord was calculates earlier

print(p_altaz.alt.deg)
print(lmc_altaz.alt.deg)

27.449051339528904
-44.01734541951676
